In [2]:
import requests
from bs4 import BeautifulSoup
import re

def get_url(url):
    """Get parsed HTML from url
      Input: url to the webpage
      Output: Parsed HTML text of the webpage
    """
    # Send Get request
    r = requests.get(url)

    # Parse HTML text
    soup = BeautifulSoup(r.text, 'html.parser')
    
    return soup

#soup = get_url()

# select the tag that contains the product


def scrape_tiki(url='https://tiki.vn/laptop-may-vi-tinh-linh-kien/c1846?src=c.1846.hamburger_menu_fly_out_banner&_lc=Vk4wMzkwMTIwMDI%3D'):
    """Scrape info of products in Laptop-PC"""
    
    # Get parsed HTML by calling get_url function
    soup = get_url(url)
           
    # Find all products
    products = soup.find_all('div', {'class':'product-item'})
    
    # Store product info in a list
    data = []
    
    count = 0
    
    # Extract info of each product
    for product in products:    
        d = {'Category':'',
         'Name':'',
         'Url':'',
         'Image_url':'',
         'Final_price':'',
         'Regular_price':'',
         'Discount':'',
         'Installment':'',
         'Cross_border':'',
         'Sponsor':'',
         'Reviews':'',
         'Rating':'',
         'Rating_by_stars':''}
    
        try:
            d['Category'] = product['data-category']
            #print(d['Category'])
            d['Name'] = product['data-title']
            #print(d['Name'])
            d['Url'] = 'https://tiki.vn' + product.a['href']
            #print(d['Url'])
            d['Image_url'] = product.find('img', {'class':'product-image img-responsive'})['src']
            #print(d['Image'])
            d['Final_price'] = int(list(product.find('p', {'class':'price-sale'}).text.split())[0].replace('đ', '').replace('.', '', -1))
            #print(d['Final_Price'])
            d['Regular_price'] = int(list(product.find('p', {'class':'price-sale'}).text.split())[2].replace('đ', '').replace('.', '', -1))
            #print(d['Regular_Price'])
            d['Discount'] = int(list(product.find('p', {'class':'price-sale'}).text.split())[1].replace('-', '').replace('%',''))

            if product.find('p', {'class':'installment'}):
                installment = 'YES'
            else:
                installment = 'NO'
            d['Installment'] = installment
            #print(d['Installment'])
        
            if 'Quốc Tế' in d['Category']:
                is_cross_border = 'YES'
            else:
                is_cross_border = 'NO'
            d['Cross_border'] = is_cross_border
            #print(d['Cross_border'])
        
            if 'Tài trợ' in product.find('div', {'class':'ship-label-wrapper'}).text:
                sponsor = 'YES'
            else:
                sponsor = 'NO'
            #print(sponsor)
            
            d['Sponsor'] = sponsor
        
            try:
                d['Rating'] = int(product.find('span', {'class':'rating-content'}).span['style'][-3:-1])
                d['Rating_by_stars'] = d['Rating'] * 5 / 100
            except:
                d['Rating'] = 0
        
            if d['Rating'] == 0:
                d['Reviews'] = 0
            else:
                d['Reviews'] = int(product.find('p', {'class':'review'}).text.replace('(', '').split()[0])
            
            data.append(d)
    
        except:
            print('Cannot retrieve information')
        
    return data


print(scrape_tiki())

[{'Category': 'Laptop - Máy Vi Tính - Linh kiện/Laptop/Laptop Truyền Thống', 'Name': 'Laptop Asus ASUSPRO B9440UA-GV0495T (Core i5-8250U/ 8GB LPDDR3-2133MHz/ 256GB SSD M.2/ 14 FHD IPS/ Win10) - Hàng Chính Hãng', 'Url': 'https://tiki.vn/laptop-asus-asuspro-b9440ua-gv0495t-core-i5-8250u-8gb-lpddr3-2133mhz-256gb-ssd-m-2-14-fhd-ips-win10-hang-chinh-hang-p11007488.html?spid=11007489&src=category-page-1846&2hi=0', 'Image_url': 'https://salt.tikicdn.com/cache/280x280/ts/product/4d/b1/db/3e4ed7f0a02d017d7086bbce1d640820.jpg', 'Final_price': 17139000, 'Regular_price': 26990000, 'Discount': 36, 'Installment': 'YES', 'Cross_border': 'NO', 'Sponsor': 'NO', 'Reviews': 7, 'Rating': 86, 'Rating_by_stars': 4.3}, {'Category': 'Laptop - Máy Vi Tính - Linh kiện/Thiết Bị Văn Phòng - Thiết Bị Ngoại Vi/Màn Hình Máy Tính/Màn Hình Phổ Thông', 'Name': 'Màn Hình Máy Tính Cong Full Viền 24inch 75Hz Mới HUGON Q24 sử dụng cáp HDMI - CHUYÊN DÙNG GAMER - Văn Phòng - Học Tập - Hàng Nhập Khẩu', 'Url': 'https://tiki.vn